In [33]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import ElasticNet
from sklearn.dummy import DummyRegressor
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
from sklearn.linear_model import Ridge

In [34]:
data = pd.read_csv('../../MA_PREDICTOR/data/ma_data_car_clean.csv')

In [35]:
data.drop('month', inplace=True, axis=1)

In [36]:
data.head()

,consideration_offered,shares_acquired,shares_at_announcement,acquiror_code,target_code,target_status,acquisition_count,bidder_count,cross_border,relatedness,economic_sector_ac,business_sector_ac,economic_sector_target,business_sector_target,car_1,car_3,car_5,car_10
0,Other,full,no,55101010,55301010,others,5,1,cross_border,economic_sector,Financials,Banking & Investment Services,Financials,Insurance,-0.002947,-0.000599,-0.019639,-0.007640
1,Other,full,no,55101010,55301010,others,6,1,cross_border,economic_sector,Financials,Banking & Investment Services,Financials,Insurance,-0.002947,-0.000599,-0.019639,-0.007640
2,Other,full,no,52102050,53205020,others,22,1,cross_border,not_related,Industrials,Industrial Goods,Consumer Cyclicals,Cyclical Consumer Products,0.006169,0.000123,-0.020599,-0.001150
3,Other,full,no,54201010,56201040,others,34,1,cross_border,not_related,Consumer Non-Cyclicals,Personal & Household Products & Services,Healthcare,Pharmaceuticals & Medical Research,0.007969,0.029287,0.009896,0.028176
4,Other,full,no,52102050,53205020,others,23,1,cross_border,not_related,Industrials,Industrial Goods,Consumer Cyclicals,Cyclical Consumer Products,0.006169,0.000123,-0.020599,-0.001150


scale the acq count


In [37]:
data.columns

Index(['consideration_offered', 'shares_acquired', 'shares_at_announcement',
       'acquiror_code', 'target_code', 'target_status', 'acquisition_count',
       'bidder_count', 'cross_border', 'relatedness', 'economic_sector_ac',
       'business_sector_ac', 'economic_sector_target',
       'business_sector_target', 'car_1', 'car_3', 'car_5', 'car_10'],
      dtype='object')

In [38]:
y = data['car_1']

In [39]:
X = data[['consideration_offered', 'shares_acquired', 'shares_at_announcement',
       'acquiror_code', 'target_code', 'target_status', 'acquisition_count',
       'bidder_count', 'cross_border', 'relatedness', 'economic_sector_ac',
       'business_sector_ac', 'economic_sector_target',
       'business_sector_target']]

# Splitting of the dataset

In [40]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=0)

# Pipeline and GridSearch

In [41]:
cat_transformer = OneHotEncoder(handle_unknown='ignore')

In [42]:
num_transformer= MinMaxScaler()

In [43]:
cat_features=['consideration_offered', 'shares_acquired', 'shares_at_announcement', 'cross_border', 'relatedness',
       'economic_sector_ac', 'business_sector_ac', 'economic_sector_target', 'target_status', 
       'business_sector_target']

In [44]:
preprocessor = ColumnTransformer([
    ('num_transformer', num_transformer, ['acquisition_count']),
    ('cat_transformer', cat_transformer, cat_features)], remainder='passthrough')

## Baseline

In [45]:
baseline_model_mean = DummyRegressor(strategy="mean") 

In [46]:
cross_val_score(baseline_model_mean, X_train, y_train, cv=5, scoring='r2').mean()

-0.0005246333864007635

In [47]:
cross_val_score(baseline_model_mean, X_train, y_train, cv=5, scoring='neg_mean_squared_error').mean()

-0.0018494774855108622

## SVR

In [48]:
model= SVR()

In [49]:
pipe = Pipeline([
    ('preprocessing', preprocessor),
    ('model', model)])

In [50]:
# Old hyperparameter Grid
grid = {'model__kernel': ['linear', 'rbf'],
        'model__gamma':  [2e-15, 2e-10, 2e-5, 2, 2e3],
        'model__C': [2e-5,2e-3,2, 2e5, 2e15]}

In [54]:
# New hyperparameter Grid
grid = {'model__kernel': ['linear'],
        'model__gamma':  [2e-5, 2, 2e3],
        'model__C': [2e-5, 2, 2e5]}

In [55]:
search = GridSearchCV(pipe, 
                      grid, 
                      scoring=['neg_mean_squared_error', 'r2', 'neg_mean_absolute_error'],
                      refit='neg_mean_squared_error',
                      cv=5,
                      n_jobs=-1)

In [56]:
# Fit data to Grid Search
search.fit(X_train,y_train);

KeyboardInterrupt: 

In [ ]:
# Best score 1
search.best_score_

In [ ]:
search.best_params_